In [1]:
!pip install pymorphy2
!git clone https://github.com/nlpub/pymystem3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 781.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=5adb0d3cb477e12e71ee9fa8955be12ec9f4c6e8fbded3d10f3b9577dcaf856c
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
Cloning into 'pymystem3'...
remote: Enumerating objects: 434, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 434 (delta 4), reused 0 (delta 0), pack-reused 425
Receiving objects: 100% (434/434), 100.19 KiB | 6.26 MiB/s, done.
Resolving deltas: 100% (261/261), done.


In [2]:
import pandas as pd
import re

import nltk
import string

In [3]:
from google.colab import drive

# Монтируем Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data1 = pd.read_csv('/content/drive/My Drive/Razmetka/description.csv',delimiter=',', header=0)
data1

,Audio,Image,Title,Description
0,guitar.zip,guitar.jpg,Гитара,Гитара - струнный щипковый лютневидный инструм...
1,flute.zip,flute.jpg,Флейта,Флейта - духовой музыкальный инструмент (аэроф...
2,saxophone.zip,saxophone.jpg,Саксофон,Саксофон - духовой язычковый музыкальный инстр...
3,violin.zip,violin.jpg,Скрипка,Скрипка - струнно-смычковый музыкальный инстру...
4,clarinet.zip,clarinet.jpg,Кларнет,Кларнет - язычковый деревянный духовой музыкал...
5,trumpet.zip,trumpet.jpg,Труба,Труба - европейский духовой амбушюрный инструм...


In [5]:
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.4 MB/s eta 0:00:00


In [6]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, OWL
from urllib.parse import quote
import os

In [16]:
graph = Graph()
graph.parse("/content/drive/My Drive/Razmetka/ontology.ttl", format='turtle')
graph

<Graph identifier=N7c93e4b2bf004906a39f70e954a1c9a6 (<class 'rdflib.graph.Graph'>)>

In [17]:
print(graph.serialize(format='ttl'))

@prefix : <http://visualdataweb.org/newOntology/> .
@prefix dc1: <http://purl.org/dc/elements/1.1/#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix s: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:Clarnet a owl:Class ;
    s:label "Clarnet"@en ;
    s:subClassOf :Type .

:Description a owl:DatatypeProperty ;
    s:label "Description"@en ;
    s:domain :Appearance ;
    s:range rdf:PlainLiteral .

:Flute a owl:Class ;
    s:label "Flute"@en ;
    s:subClassOf :Type .

:Guitar a owl:Class ;
    s:label "Guitar"@en ;
    s:subClassOf :Type .

:Saxophone a owl:Class ;
    s:label "Saxophone"@en ;
    s:subClassOf :Type .

:Trumpet a owl:Class ;
    s:label "Trumpet"@en ;
    s:subClassOf :Type .

:Violin a owl:Class ;
    s:label "Violin"@en ;
    s:subClassOf :Type .

:belongsToType a owl:ObjectProperty ;
    s:label "belongsToType"@en ;
    s:domain :Instruments ;
    

In [11]:
OWL_URI = 'http://owlgred.lumii.lv/online_visualization/4r20#'
NS = Namespace(OWL_URI)

In [12]:
# Classes
instruments_class = NS.Instruments
type_class = NS.Type
action_class = NS.Action
note_class = NS.Note
appearance_class = NS.Appearance

# Note SubClasses
note_classes = {
    "fricoe": NS.FriCoe,
    "motalb": NS.MotAlb,
    "parmaj": NS.ParMaj,
    "pasdom": NS.PasDom,
    "stuvul": NS.StuVul,
    "turmer": NS.TurMer
}

# Object and Data Properties
description_prop = NS.Description
belongs_to_note_prop = NS.belongsToNote
has_action_prop = NS.hasAction
has_appearance_prop = NS.hasAppearance
typevoice_prop = NS.TypeVoice
is_carrying_prop = NS.isCarrying
is_feeding_prop = NS.isFeeding
is_flying_prop = NS.isFlying
is_sitting_prop = NS.isSitting
name_ru_prop = NS.nameRu
file_name_prop = NS.fileName

In [ ]:

data_df = pd.read_csv("/content/drive/My Drive/Razmetka/description.csv")
for index, row in data_df.iterrows():
    instruments_instance_uri = URIRef(OWL_URI + row['name (lat)'].replace(' ', '_'))
    note_class_uri = note_classes.get(row['note'])

    # Добавление инструмента
    graph.add((instruments_instance_uri, RDF.type, instruments_class))
    graph.add((instruments_instance_uri, belongs_to_note_prop, Literal(row['note'])))
    graph.add((instruments_instance_uri, name_ru_prop, Literal(row['name (ru)'], lang='ru')))


In [ ]:
audio_files = os.listdir("/content/drive/My Drive/Razmetka/description./instruments_audio")
for audio_file in audio_files:
    note_name = os.path.splitext(audio_file)[0].split('_')[0]
    instruments_instance_uri = URIRef(OWL_URI + note_name)

    # Создание записи голоса
    type_instance_uri = URIRef(OWL_URI + note_name + "_type")
    graph.add((type_instance_uri, RDF.type, type_class))
    graph.add((instruments_instance_uri, has_type_prop, type_instance_uri))

In [ ]:
!pip install tabulate
from tabulate import tabulate

In [ ]:
query = """
PREFIX ns: <http://www.semanticweb.org/owl/owlapi/turtle#>

SELECT ?instruments ?nameRu ?description WHERE {
    ?instruments rdf:type ns:Instruments .
    ?instruments ns:nameRu ?nameRu .
    ?instruments ns:hasAppearance ?appearance .
    ?appearance ns:Description ?description .
}
"""

# Выполнение запроса
results = graph.query(query)

table = []
for row in results:
    table.append([str(row.nameRu), str(row.description)])

headers = ["Name (RU)", "Description"]
print(tabulate(table, headers, tablefmt="grid"))